## Ingeniería de variables

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.impute import KNNImputer, IterativeImputer
from sklearn.linear_model import LinearRegression
from sklearn.experimental import enable_iterative_imputer

In [ ]:
def plot_percent_cols(df, li=25, lm=50, ls=75):
    # Calcular los porcentajes de datos nulos y no nulos
    null_percent = df.isnull().mean() * 100
    not_null_percent = 100 - null_percent

    # Crear la figura y el eje con un tamaño específico
    fig, ax = plt.subplots(figsize=(10, 6))

    # Crear el gráfico de barras acumulado
    ax.bar(df.columns, not_null_percent, label='Not Null')
    ax.bar(df.columns, null_percent, bottom=not_null_percent, label='Null')

    # Agregar las líneas horizontales
    ax.axhline(li, color='r', linestyle='--')
    ax.axhline(lm, color='g', linestyle='--')
    ax.axhline(ls, color='b', linestyle='--')

    # Agregar la leyenda
    ax.legend()

    # Agregar títulos
    plt.title('Porcentaje de datos nulos y no nulos por columna')

    # Mostrar el gráfico
    plt.xticks(rotation=90)
    plt.show()

### 1. Leer los datos

In [ ]:
df = pd.read_parquet('./data/1_data_exploracion_datos.parquet')
plot_percent_cols(df)

### 2. Limpieza general de datos

In [ ]:
# Eliminar las columnas con más del 50% de datos nulos
df = df.dropna(thresh=df.shape[0]*0.5, axis=1)
plot_percent_cols(df, li=70, lm=80, ls=90)

### 3. Limpieza y tratamiento de datos categóricos

In [ ]:
# Seleccionar las columnas con datos categoricos
df_cat = df.select_dtypes(include=['object'])
plot_percent_cols(df_cat, li=70, lm=80, ls=90)

In [ ]:
# Eliminar la columna PROVINCIA, FEC_LLAMADA y DF_TYPE
df_cat = df_cat.drop(columns=['PROVINCIA', 'FEC_LLAMADA', 'DF_TYPE'])
print('Eliminando columnas PROVINCIA, FEC_LLAMADA y DF_TYPE')

In [ ]:
# Reducir a 2 categorias la columna DEPARTAMENTO
df_cat['DEPARTAMENTO'] = df_cat['DEPARTAMENTO'].apply(lambda x: 'FUERA DE LIMA' if x != 'LIMA' else x)
print(df_cat['DEPARTAMENTO'].value_counts(dropna=False, normalize=True))

In [ ]:
# Crear categoría "SIN DATOS" para FBK_ULT6, FBK_ULT12, FBK_BEST6, FBK_BEST12
df_cat['FBK_ULT6'] = df_cat['FBK_ULT6'].fillna('SIN DATOS')
df_cat['FBK_ULT12'] = df_cat['FBK_ULT12'].fillna('SIN DATOS')
df_cat['FBK_BEST6'] = df_cat['FBK_BEST6'].fillna('SIN DATOS')
df_cat['FBK_BEST12'] = df_cat['FBK_BEST12'].fillna('SIN DATOS')

In [ ]:
# Imputar moda para la columna COD_SALA con datos nulos
df_cat['COD_SALA'] = df_cat['COD_SALA'].fillna(df_cat['COD_SALA'].mode()[0])
print(df_cat['COD_SALA'].value_counts(dropna=False, normalize=True))

In [ ]:
# Imputar moda para la columna SEGMENTO con datos nulos
df_cat['SEGMENTO'] = df_cat['SEGMENTO'].fillna(df_cat['SEGMENTO'].mode()[0])
print(df_cat['SEGMENTO'].value_counts(dropna=False, normalize=True))

In [ ]:
# Imputar moda para la columna RANGO_INGRESOS con datos nulos
df_cat['RANGO_INGRESOS'] = df_cat['RANGO_INGRESOS'].fillna(df_cat['RANGO_INGRESOS'].mode()[0])
print(df_cat['RANGO_INGRESOS'].value_counts(dropna=False, normalize=True))

In [ ]:
# Eliminar FBK_ULT6, FBK_BEST6
df_cat = df_cat.drop(columns=['FBK_ULT6', 'FBK_BEST6'])
print('Eliminando columnas')

In [ ]:
sns.heatmap(df_cat.isnull(), cbar=False)

In [ ]:
df.shape[0] == df_cat.shape[0]

#### 4. Limpieza y tratamiendo de datos numéricos

In [ ]:
# Seleccionar las columnas con datos númericos
df_num = df.select_dtypes(include=['float64', 'int64'])

In [ ]:
# Eliminar columnas
df_num = df_num.drop(columns=['IDGRUPO', 'INGRESO_BRUTO', 'INGRESO_NETO_VIGENTE', 'TARGET', 'DIAS_ACT'])
print('Eliminando columnas')

In [ ]:
plot_percent_cols(df_num, li=55, lm=60, ls=70)

In [ ]:
# Graficar la correlación de columnas númericas
plt.figure(figsize=(10, 6))
sns.heatmap(df_num.corr(), annot=True, cmap='coolwarm')

In [ ]:
# Eliminar columnas con correlación mayor a 0.9
corr = df_num.corr().abs()
upper = corr.where(np.triu(np.ones(corr.shape), k=1).astype(bool))
to_drop = [column for column in upper.columns if any(upper[column] > 0.90)]
df_num = df_num.drop(columns=to_drop)
print('Eliminando columnas')

In [ ]:
plt.figure(figsize=(10, 6))
sns.heatmap(df_num.corr(), annot=True, cmap='coolwarm')

In [ ]:
plot_percent_cols(df_num, li=55, lm=60, ls=70)

In [ ]:
# Eliminar DIAS_BEST6
df_num = df_num.drop(columns=['DIAS_BEST6'])
plot_percent_cols(df_num, li=55, lm=60, ls=70)

In [ ]:
# NC_DIAS6: Imputar la mediana
df_num['NC_DIAS6'] = df_num['NC_DIAS6'].fillna(df_num['NC_DIAS6'].median())
plot_percent_cols(df_num, li=55, lm=60, ls=70)

In [ ]:
# NC_CTD12: Imputar la mediana
df_num['NC_CTD12'] = df_num['NC_CTD12'].fillna(df_num['NC_CTD12'].median())
plot_percent_cols(df_num, li=55, lm=60, ls=70)

In [ ]:
# DIAS_BEST12: Imputar con KNNImputer
imputer = KNNImputer(n_neighbors=5)
df_num['DIAS_BEST12'] = imputer.fit_transform(df_num[['DIAS_BEST12']])
plot_percent_cols(df_num, li=55, lm=60, ls=70)